In [47]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import tweepy
import os
from dotenv import load_dotenv
import tempfile
from datetime import datetime
import pytz
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
import html
import pandas as pd
from duckduckgo_search import DDGS
import re
from Levenshtein import ratio
import logging
import time
from requests.exceptions import RequestException
import sqlite3
import urllib.parse

def get_steam_page_info(app_id):
    url = f"https://store.steampowered.com/app/{app_id}/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Vérifier la présence de contenu généré par IA
        ai_disclosure = soup.find(string=re.compile("AI GENERATED CONTENT DISCLOSURE", re.IGNORECASE))
        
        # Récupérer les tags
        tag_elements = soup.find_all('a', class_='app_tag')
        tags = [tag.text.strip() for tag in tag_elements][:4]  # Limiter à 4 tags
        
        # Récupérer le lien Twitter s'il existe
        twitter_link = soup.find('a', class_="ttip", attrs={'data-tooltip-text': lambda x: x and 'x.com/' in x})
        twitter_handle = None
        if twitter_link:
            twitter_url = twitter_link['data-tooltip-text']
            twitter_handle = '@' + twitter_url.split('/')[-1]

        
        return {
            'ai_generated': bool(ai_disclosure),
            'tags': tags,
            'twitter_handle': twitter_handle
        }
    except Exception as e:
        logging.error(f"Erreur lors du scraping pour le jeu {app_id}: {e}")
        return None
    
get_steam_page_info(3075040)
 

{'ai_generated': False,
 'tags': ['Casual', 'Cooking', 'Strategy', 'Cute'],
 'twitter_handle': '@SasanquaGames'}

In [22]:
res = get_steam_page_info(3075040)
res

{'ai_generated': False,
 'tags': ['Casual', 'Cooking', 'Strategy', 'Cute'],
 'twitter_handle': None}

In [36]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
import logging


url = f"https://store.steampowered.com/app/3075040/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')

soup


<!DOCTYPE html>

<html class="responsive" lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="#171a21" name="theme-color"/>
<title>EggGarden on Steam</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://store.akamai.steamstatic.com/public/shared/css/motiva_sans.css?v=Rc2hpzg2Ex3T&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/shared/css/shared_global.css?v=GJRG1UROmBaQ&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/shared/css/buttons.css?v=CrrkDubPqLcq&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/css/v6/store.css?v=Faa1tVDQ0orR&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/shared/css/user_rev

In [45]:
soup.find('a', class_="ttip", attrs={'data-tooltip-text': lambda x: x and 'x.com/' in x})

#soup.find_all("a", class_="ttip")

<a class="ttip" data-tooltip-text="https://x.com/SasanquaGames" href="https://steamcommunity.com/linkfilter/?u=https%3A%2F%2Fx.com%2FSasanquaGames" rel="noopener" target="_blank">
<img src="https://store.akamai.steamstatic.com/public/images/v6/social/x_32_white.png"/>
<span class="social_account">X (Twitter)</span>
<img align="bottom" border="0" src="https://store.akamai.steamstatic.com/public/images/v5/ico_external_link.gif"/>
</a>